<a href="https://colab.research.google.com/github/JonasZaoui/test/blob/main/modelJONAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torchio==0.18.75
!pip install -q monai==0.8.1
!pip install -q pytorch-lightning==1.5.10
!pip install -q gdown==3.6.4 matplotlib==3.2.2 pandas==1.1.5 seaborn==0.11.1

     |████████████████████████████████| 164 kB 5.4 MB/s 
     |████████████████████████████████| 48.4 MB 2.0 MB/s 
     |████████████████████████████████| 721 kB 5.4 MB/s 
     |████████████████████████████████| 527 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 952 kB 39.0 MB/s 
     |████████████████████████████████| 408 kB 46.9 MB/s 
     |████████████████████████████████| 136 kB 39.2 MB/s 
     |████████████████████████████████| 829 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 271 kB 53.3 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 50.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which i

In [98]:
import monai 
import matplotlib.pyplot as plt
import torchio as tio
import time
import pandas as pd
from datetime import datetime
import torch 
import torchvision
import pytorch_lightning as pl
from pathlib import Path
from torch.utils.data import random_split, DataLoader


plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()

print('Last run on', time.ctime())


Last run on Fri Apr 29 13:15:33 2022


In [148]:
class Dataset(pl.LightningDataModule):
    def __init__(self,dataset_dir,batch_size,train_val_ratio):
        super().__init__()
        self.batch_size = batch_size
        self.dataset_dir = Path(dataset_dir)
        self.subjects = None
        self.train_val_ratio = train_val_ratio
        self.test_subjects = None
        self.preprocess = None
        self.transform = None
        self.train_set = None
        self.val_set = None
        self.test_set = None

    
    def download_data(self):
       
        def get_niis(d):
            return sorted(p for p in d.glob('*.nii*') if not p.name.startswith('.'))

        image_training_paths = get_niis(self.dataset_dir / 'mri')
        label_training_paths = get_niis(self.dataset_dir / 'labels')
        return image_training_paths, label_training_paths

    def prepare_data(self):
        image_training_paths, label_training_paths = self.download_data()

        self.subjects = []
        for image_path, label_path in sorted(zip(image_training_paths, label_training_paths)):
            subject = tio.Subject(
                image=tio.ScalarImage(image_path),
                label=tio.LabelMap(label_path)
            )
            self.subjects.append(subject)

        
    
    def get_preprocessing_transform(self):
        preprocess = tio.RescaleIntensity((-1, 1))
        return preprocess
    
    def get_augmentation_transform(self):
        augment = tio.RandomGamma(p=0.5)
        return augment

    def setup(self, stage=None):
        
        num_subjects = len(self.subjects)
        num_train_subjects = int(round(num_subjects * self.train_val_ratio))
        num_val_subjects = num_subjects - num_train_subjects
        splits = num_train_subjects, num_val_subjects
        train_subjects, val_subjects = random_split(self.subjects, splits)

        self.preprocess = self.get_preprocessing_transform()
        augment = self.get_augmentation_transform()
        self.transform = tio.Compose([self.preprocess, augment])

        self.train_set = tio.SubjectsDataset(train_subjects, transform=self.transform) #data augmentation only on the trainset. 
        self.val_set = tio.SubjectsDataset(val_subjects, transform=self.preprocess)

        

    def train_dataloader(self):
        patches_queue_train = tio.Queue(self.train_set,max_length=14,samples_per_volume=7,sampler=tio.data.UniformSampler(patch_size=(144,144,32)),num_workers=2) #!!!!!!!!!!! sampler is correct ? 
        patches_loader_train = DataLoader(patches_queue_train,self.batch_size,num_workers=0)  # this must be 0 #!!!!!!!!!!!

        # for one_batch in patches_loader_train:

        #   # From: [batch_size, depth, height, width, channels]
        #   # To: [batch_size, channels, depth, height, width]

        #   one_batch['image'][tio.DATA] = torch.permute(one_batch['image'][tio.DATA], (0, 4, 1, 2, 3))
        #   one_batch['label'][tio.DATA] = torch.permute(one_batch['label'][tio.DATA], (0, 4, 1, 2, 3))


        return patches_loader_train

    #torchio.data.WeightedSampler ne marche pas si le masque de probabilités est nul -> si label ne contient pas de microbleeds. Cette fonction associe les valeurs des pixels a des probas. 

    def val_dataloader(self):
        patches_queue_val = tio.Queue(self.val_set,max_length=14,samples_per_volume=7,sampler=tio.data.UniformSampler(patch_size=(144,144,32)),num_workers=2) #!!!!!!!!!!! sampler is correct ? 
        patches_loader_val = DataLoader(patches_queue_val,self.batch_size,num_workers=0)  # this must be 0 #!!!!!!!!!!!

        # for one_batch in patches_loader_val:

        #   # From: [batch_size, depth, height, width, channels]
        #   # To: [batch_size, channels, depth, height, width]
        #   one_batch['image'][tio.DATA] = torch.permute(one_batch['image'][tio.DATA], (0, 4, 1, 2, 3))
        #   one_batch['label'][tio.DATA] = torch.permute(one_batch['label'][tio.DATA], (0, 4, 1, 2, 3))
          

        return patches_loader_val

In [149]:
data = Dataset(
    dataset_dir = '/content/drive/MyDrive/test/train_positif',
    batch_size=1, #Batch SIze de 4 ? 
    train_val_ratio=0.8
)

In [150]:
data.prepare_data()
data.setup()
print('Training:  ', len(data.train_set))
print('Validation: ', len(data.val_set))

Training:   7
Validation:  2


In [151]:
loader_train = data.train_dataloader()
one_batch = next(iter(loader_train))


In [152]:
print(one_batch['image'][tio.DATA].size())
print(one_batch['label'][tio.DATA].size())

torch.Size([1, 1, 144, 144, 32])
torch.Size([1, 1, 144, 144, 32])


In [154]:
for patch in loader_train:
      patch['image'][tio.DATA] =torch.permute(patch['image'][tio.DATA], (0, 4, 1, 2, 3))
      patch['label'][tio.DATA] =torch.permute(patch['label'][tio.DATA], (0, 4, 1, 2, 3))
      


In [159]:
one_batch = next(iter(loader_train))
print(one_batch['image'][tio.DATA].size())


torch.Size([1, 1, 144, 144, 32])


In [ ]:
#Visualisation d'un batch 

# from IPython import display

# training_set = data.train_dataloader()
# one_batch = next(iter(training_set))
# k=24  
# batch_mri = one_batch['image'][tio.DATA][..., k]
# batch_label = one_batch['label'][tio.DATA][..., k]
# slices = torch.cat((batch_mri, batch_label))
# image_path = 'batch_whole_images.png'
# torchvision.utils.save_image(
# slices,
# image_path,
# normalize=True,
# scale_each=True,
# padding=0,)

# display.Image(image_path)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


FileNotFoundError: ignored

In [125]:
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate #pas d'apprentissage pour optimiseur
        self.net = net #model utilisé 
        self.criterion = criterion #fonction cout a minimiser
        self.optimizer_class = optimizer_class #Optimiseur choisi
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr) #On applique l'optimiseur aux paramètres du modele
        return optimizer
    
    def prepare_batch(self, batch):
        return batch['image'][tio.DATA], batch['label'][tio.DATA]
    
    def infer_batch(self, batch): # calcul l'erreur entre la sortie du model et le label. 
        x, y = self.prepare_batch(batch) 
        y_hat = self.net(x) #prediction
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch) # recupere la prediction et le label
        loss = self.criterion(y_hat, y) #calcul la perte entre les 2
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
        
    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss

In [141]:
#les patchs sont en 144x144x32
#les images sont en 208x208x48

unet = monai.networks.nets.UNet(
    dimensions=3,
    in_channels=32, #nb de channels d'un patch #!!!!!!!!!!!
    out_channels=1, #Car sortie binaire #!!!!!!!!!!!
    channels=(8, 16, 32, 64),
    strides=(2, 2, 2),
)

model = Model(
    net=unet, #model utilisé
    criterion=monai.losses.FocalLoss(gamma=2.0), #fonction cout  --> pas de softmax ?? #!!!!!!!!!!!
    learning_rate=1e-2, #pas d'apprentissage 
    optimizer_class=torch.optim.AdamW, #optimiseur
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)

#gpus=1 ? 
trainer = pl.Trainer( 
    gpus=1, 
    precision=16, #!!!!!!!!!!!
    callbacks=[early_stopping],
)
trainer.logger._default_hp_metric = False

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [142]:
start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | net       | UNet      | 128 K 
1 | criterion | FocalLoss | 0     
----------------------------------------
128 K     Trainable params
0         Non-trainable params
128 K     Total params
0.257     Total estimated model params size (MB)


Training started at 2022-04-29 13:43:20.732490


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: ignored

In [128]:
# RuntimeError: Given groups=1, weight of size [8, 48, 3, 3, 3], expected input[1, 1, 144, 144, 32] to have 48 channels, but got 1 channels instead


        #   # From: [batch_size, depth, height, width, channels]
        #   # To: [batch_size, channels, depth, height, width]

        #   one_batch['image'][tio.DATA] = torch.permute(one_batch['image'][tio.DATA], (0, 4, 1, 2, 3))
        #   one_batch['label'][tio.DATA] = torch.permute(one_batch['label'][tio.DATA], (0, 4, 1, 2, 3))